In [1]:
import argparse
import os
import sys
import time
import numpy as np

from PIL import Image

import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import urllib
import time

In [2]:
import torch
from functools import partial
from torch import nn, einsum
from einops import repeat, rearrange

In [3]:
def exists(val):
    return val is not Nonev

In [4]:
def map_fn(fn, *args, **kwargs):
    def inner(*arr):
        return map(lambda t: fn(t, *args, **kwargs), arr)
    return inner

In [5]:
def safe_cat_tf(t, el, dim =0):
    if not exists(t):
        return el
    return tf.concat((t, el), axis = dim)

In [6]:
def safe_cat_pt(t, el, dim = 0):
    if not exists(t):
        return el
    return torch.cat((t, el), dim = dim) #concat

# tests

In [7]:
value_dim = 64
key_dim = 64
hidden_dim = 512
output_dim = 4
encoder = None

In [110]:
testA1 = tf.ones((3, 10,2))
testA1

<tf.Tensor: shape=(3, 10, 2), dtype=float32, numpy=
array([[[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]]], dtype=float32)>

In [123]:
testA2 = tf.zeros((3,2))
testA2 = tf.keras.layers.Reshape((1,2))(testA2)
testA2

<tf.Tensor: shape=(3, 1, 2), dtype=float32, numpy=
array([[[0., 0.]],

       [[0., 0.]],

       [[0., 0.]]], dtype=float32)>

In [124]:
tfC = tf.keras.layers.concatenate([testA1,testA2], axis=1)
tfC

<tf.Tensor: shape=(3, 11, 2), dtype=float32, numpy=
array([[[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [0., 0.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [0., 0.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [0., 0.]]], dtype=float32)>

In [10]:
tfWk = 2
tfWk * tfC

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.],
       [2., 2., 4., 4.]], dtype=float32)>

In [11]:
tf.math.reduce_sum(tfWk * tfC, axis = 1)

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([12., 12., 12., 12., 12., 12., 12., 12., 12., 12.], dtype=float32)>

In [12]:
c = tf.constant(1.5, shape=(10, 1))
c * tfC

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ],
       [1.5, 1.5, 3. , 3. ]], dtype=float32)>

In [93]:
torchA1 = torch.ones((3, 10,2))
torchA2 = torch.full((3,2),2.0)
torchA1

tensor([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]])

In [94]:
torchA2

tensor([[2., 2.],
        [2., 2.],
        [2., 2.]])

In [97]:
torchC = torch.cat((torchA1,torchA2), dim=-1)
torchC

RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [15]:
torchWk = 2
torchWk * torchC

tensor([[2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.],
        [2., 2., 4., 4.]])

In [16]:
(torchWk * torchC).sum(dim = 1)

tensor([12., 12., 12., 12., 12., 12., 12., 12., 12., 12.])

In [17]:
twkt = torch.full((10,1),1.5)
twkt * torchC

tensor([[1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000],
        [1.5000, 1.5000, 3.0000, 3.0000]])

In [18]:
tlist = []

for i in range(0, 4):
    tlist.append(torch.full((10,1),i, dtype=float))
tlist

[tensor([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]], dtype=torch.float64),
 tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]], dtype=torch.float64),
 tensor([[2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.]], dtype=torch.float64),
 tensor([[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.]], dtype=torch.float64)]

In [19]:
torch.stack(tlist)

tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.]],

        [[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [3.]]], dtype=torch.float64)

In [20]:
tflist = []

for i in range(0,5):
    tflist.append(tf.constant(i, shape=(10, 1), dtype=float))
    
tflist

[<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
      

In [21]:
tf.stack(tflist)

<tf.Tensor: shape=(5, 10, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]],

       [[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]],

       [[3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.]],

       [[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.]]], dtype=float32)>

# img test

In [22]:
allI = []
for i in range(3):
    img_fname = '../imgs/' + str(i) + '.png'
    img = torch.Tensor(np.array(Image.open(img_fname))) / 255.
    allI.append(img)
allI = torch.stack(allI, 0)
allI.shape

torch.Size([3, 32, 32])

In [23]:
tfimg = tf.convert_to_tensor(np.array(Image.open(img_fname)) / 255.)
tfimg.shape

TensorShape([32, 32])

# Einsum test

In [26]:
tf_test_1 = np.random.uniform(size=(1,32,32))
tf_test_2 = np.random.uniform(size=(1,32))
tf_test1.shape

NameError: name 'tf_test1' is not defined

In [27]:
sim = tf.einsum('b n d, b d -> b n',tf_test_1, tf_test_2)
sim                                      

<tf.Tensor: shape=(1, 32), dtype=float64, numpy=
array([[10.02943939,  8.14799901,  8.72288361,  7.65564583,  7.98988   ,
         9.76207988,  9.56242772,  9.50701888,  9.40607796, 10.31015884,
         9.07277309,  9.94455877, 10.25403429,  9.11970804, 11.34382705,
        11.40559457,  9.91675205,  9.19788983, 10.86410939, 11.12031193,
         9.96533638,  9.06565231,  9.13012947,  8.45070044,  8.24349484,
        11.01577088, 10.5269025 ,  6.99731903, 10.20250775,  9.6150974 ,
        10.43246785,  7.99250554]])>

In [28]:
torch.einsum('b n d, b d -> b n',
             torch.from_numpy(tf_test_1), 
             torch.from_numpy(tf_test_2))

tensor([[10.0294,  8.1480,  8.7229,  7.6556,  7.9899,  9.7621,  9.5624,  9.5070,
          9.4061, 10.3102,  9.0728,  9.9446, 10.2540,  9.1197, 11.3438, 11.4056,
          9.9168,  9.1979, 10.8641, 11.1203,  9.9653,  9.0657,  9.1301,  8.4507,
          8.2435, 11.0158, 10.5269,  6.9973, 10.2025,  9.6151, 10.4325,  7.9925]],
       dtype=torch.float64)

# TF Array TEst

In [36]:
out = tf.TensorArray(dtype=tf.float32,
                             size=2,
                             dynamic_size=True)

In [37]:
out.size()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [38]:
out.write(0, (1,1))

In [39]:
out.write(1,(2,2))

In [40]:
out.size()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [32]:
#out.stack()

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [2., 2.]], dtype=float32)>

# loop test

In [64]:
ind = tf.constant(0)
ind

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [63]:
def body(ind):
    ind = tf.add(ind,1)
    return([ind])
body(ind)

[<tf.Tensor: shape=(1, 1, 1, 1), dtype=int32, numpy=array([[[[1]]]], dtype=int32)>]

In [62]:
tf.while_loop(lambda i,ind: i < 3, body, [ind])

TypeError: <lambda>() missing 1 required positional argument: 'ind'

In [75]:
def cond(l, iters):
    return tf.less(l, iters)

def body(l, iters):
    print("In loop ", l)
    return [tf.add(l, 1), iters]

# t1 = tf.constant(1)
# t2 = tf.constant(5)
iters = tf.constant(3)

l, iters = tf.while_loop(cond, body, [0, iters])

tf.get_static_value(l)

In loop  0
In loop  tf.Tensor(1, shape=(), dtype=int32)
In loop  tf.Tensor(2, shape=(), dtype=int32)


3

# .mean

In [127]:
t1 = torch.ones((10,256,128))
t1.shape

torch.Size([10, 256, 128])

In [133]:
t1.mean(1).shape

torch.Size([10, 128])

In [131]:
t2 = tf.ones((10,256,128))
t2.shape

TensorShape([10, 256, 128])

In [136]:
tf.reduce_mean(t2, axis=1).shape

TensorShape([10, 128])

# .var

In [236]:
list = []
list.append(torch.zeros(128))
list.append(torch.ones(128))
z_seq = torch.stack(list, dim=0)
z_seq.var(0)

tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 

# positional Encodings

In [140]:
max_len = 5000
d_model = 256
tf.zeros((max_len, d_model)).shape

TensorShape([5000, 256])

In [199]:
pe1 = torch.zeros(max_len, d_model)
pe1.shape

torch.Size([5000, 256])

In [198]:
pe2 = np.zeros((max_len, d_model))
pe2.shape

(5000, 256)

In [182]:
position1 = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
position1

tensor([[0.0000e+00],
        [1.0000e+00],
        [2.0000e+00],
        ...,
        [4.9970e+03],
        [4.9980e+03],
        [4.9990e+03]])

In [183]:
position2 = tf.expand_dims(tf.range(0, max_len, dtype=tf.float32),axis=1)
position2

<tf.Tensor: shape=(5000, 1), dtype=float32, numpy=
array([[0.000e+00],
       [1.000e+00],
       [2.000e+00],
       ...,
       [4.997e+03],
       [4.998e+03],
       [4.999e+03]], dtype=float32)>

In [184]:
import math

In [185]:
div_term1 = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * (-math.log(10000.0) / d_model))
div_term1

tensor([1.0000e+00, 9.3057e-01, 8.6596e-01, 8.0584e-01, 7.4989e-01, 6.9783e-01,
        6.4938e-01, 6.0430e-01, 5.6234e-01, 5.2330e-01, 4.8697e-01, 4.5316e-01,
        4.2170e-01, 3.9242e-01, 3.6517e-01, 3.3982e-01, 3.1623e-01, 2.9427e-01,
        2.7384e-01, 2.5483e-01, 2.3714e-01, 2.2067e-01, 2.0535e-01, 1.9110e-01,
        1.7783e-01, 1.6548e-01, 1.5399e-01, 1.4330e-01, 1.3335e-01, 1.2409e-01,
        1.1548e-01, 1.0746e-01, 1.0000e-01, 9.3057e-02, 8.6596e-02, 8.0584e-02,
        7.4989e-02, 6.9783e-02, 6.4938e-02, 6.0430e-02, 5.6234e-02, 5.2330e-02,
        4.8697e-02, 4.5316e-02, 4.2170e-02, 3.9242e-02, 3.6517e-02, 3.3982e-02,
        3.1623e-02, 2.9427e-02, 2.7384e-02, 2.5483e-02, 2.3714e-02, 2.2067e-02,
        2.0535e-02, 1.9110e-02, 1.7783e-02, 1.6548e-02, 1.5399e-02, 1.4330e-02,
        1.3335e-02, 1.2409e-02, 1.1548e-02, 1.0746e-02, 1.0000e-02, 9.3057e-03,
        8.6596e-03, 8.0584e-03, 7.4989e-03, 6.9783e-03, 6.4938e-03, 6.0430e-03,
        5.6234e-03, 5.2330e-03, 4.8697e-

In [186]:
div_term2 = tf.exp(tf.range(0, d_model, 2, dtype=tf.float32) * (-math.log(10000.0) / d_model))
div_term2

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([1.00000000e+00, 9.30572033e-01, 8.65964293e-01, 8.05842221e-01,
       7.49894202e-01, 6.97830617e-01, 6.49381638e-01, 6.04296386e-01,
       5.62341332e-01, 5.23299098e-01, 4.86967504e-01, 4.53158379e-01,
       4.21696484e-01, 3.92418951e-01, 3.65174145e-01, 3.39820832e-01,
       3.16227764e-01, 2.94272721e-01, 2.73841947e-01, 2.54829645e-01,
       2.37137347e-01, 2.20673412e-01, 2.05352500e-01, 1.91095293e-01,
       1.77827939e-01, 1.65481701e-01, 1.53992638e-01, 1.43301249e-01,
       1.33352146e-01, 1.24093764e-01, 1.15478203e-01, 1.07460767e-01,
       9.99999940e-02, 9.30572078e-02, 8.65964293e-02, 8.05842206e-02,
       7.49894157e-02, 6.97830617e-02, 6.49381578e-02, 6.04296364e-02,
       5.62341250e-02, 5.23299091e-02, 4.86967564e-02, 4.53158319e-02,
       4.21696529e-02, 3.92418914e-02, 3.65174115e-02, 3.39820795e-02,
       3.16227749e-02, 2.94272732e-02, 2.73841936e-02, 2.54829675e-02,
       2.37137340e-02, 2.2067

In [201]:
pe1[:, 0::2] = torch.sin(position1 * div_term1)
pe1[:, 1::2] = torch.cos(position1 * div_term1)
pe1.shape

torch.Size([5000, 256])

In [202]:
pe2[:, 0::2] = tf.sin(position2 * div_term2)
pe2[:, 1::2] = tf.cos(position2 * div_term2)
pe2.shape

(5000, 256)

In [208]:
pe1.unsqueeze(0).transpose(0,1).shape

torch.Size([5000, 1, 256])

In [210]:
np.transpose(np.expand_dims(pe2,axis=0),(1,0,2)).shape

(5000, 1, 256)